# Circuit from YAML

Sometimes it’s useful to be able to define circuits from YAML definitions. To not re-invent the wheel, SAX uses GDSFactory’s YAML netlist spec to define its circuits. This makes it very easy to convert a GDSFactory layout to a SAX circuit model!

In [ ]:
import sax
import jax.numpy as jnp
import matplotlib.pyplot as plt

## MZI

Let’s first see how we can define a SAX circuit from YAML:

In [ ]:
netlist = """
instances:
    lft:
      component: coupler
      settings:
          coupling: 0.5
    rgt:
        component: coupler
        settings:
            coupling: 0.5
    top:
        component: straight
        settings:
            length: 25.0
    btm:
        component: straight
        settings:
            length: 15.0

connections:
    lft,out0: btm,in0
    btm,out0: rgt,in0
    lft,out1: top,in0
    top,out0: rgt,in1

ports:
    in0:  lft,in0
    in1:  lft,in1
    out0: rgt,out0
    out1: rgt,out1

"""

In [ ]:
mzi = sax.circuit_from_yaml(netlist)

In [ ]:
wl = jnp.linspace(1.5, 1.6, 1000)
params = sax.get_params(mzi)
params = sax.set_params(params, wl=wl)
transmission = jnp.abs(mzi(**params)["in0", "out0"]) ** 2

plt.plot(wl * 1e3, transmission)
plt.xlabel("λ [nm]")
plt.ylabel("T")
plt.show()

That was easy! However, during the above YAML conversion, only models available in sax.models were used. What if we want to map the YAML component names to custom models? Let’s say we want to use a dispersionless waveguide for the above model for example:

In [ ]:
def waveguide_without_dispersion(wl=1.55, length=25.0, neff=2.34):
    phase = 2 * jnp.pi * neff * length / wl
    sdict = sax.reciprocal({("in0", "out0"): jnp.exp(1j * phase)})
    return sdict

We can regenerate the above circuit again, but this time we specify a models mapping:

In [ ]:
mzi = sax.circuit_from_yaml(netlist, models={"straight": waveguide_without_dispersion})

The models= keyword in circuit_from_yaml can be a dictionary or an imported python module (like for example sax.models). Or a list containing multiple of such dictionary mappings and imported modules.

In [ ]:
wl = jnp.linspace(1.5, 1.6, 1000)
params = sax.get_params(mzi)
params = sax.set_params(params, wl=wl)
transmission = jnp.abs(mzi(**params)["in0", "out0"]) ** 2

plt.plot(wl, transmission)
plt.xlabel("Wavelength [nm]")
plt.ylabel("T")
plt.show()


## Compatibility with GDSFactory

GDSFactory is an python-based photonic layout tool. 
We can now use the above introduced yaml definition as a translation layer between SAX and GDSFactory:

Let’s start from GDSFactory’s default mzi:

In [ ]:
import gdsfactory as gf
mzi_component = gf.components.mzi()
mzi_component

In [ ]:
netlist = mzi_component.get_netlist_yaml()
print(netlist)

As you can see from the yaml netlist, there are 3 different kinds of components in this circuit:

-   straight
-   bend_euler
-   mmi1x2

we have a ‘straight’ model in sax.models, but the other two will need to be mapped. Moreover, the port conventions in GDSFactory are different. Let’s first remap the ports of our straight model to match the port-convention in GDSFactory:

In [ ]:
netlist = mzi_component.get_netlist_yaml()

In [ ]:
straight = sax.rename_ports(sax.models.straight, {"in0": "W0", "out0": "E0"})

Let’s create a model for bend_euler. Let’s stub in a dummy model for now:

In [ ]:
def bend_euler(radius=1.0, length=10.0):
    return sax.reciprocal({("W0", "N0"): 0.95})

Let’s also create a model for a 1x2 MMI:

In [ ]:
def mmi1x2(length=3.0):
    sdict = sax.reciprocal(
        {
            ("W0", "E0"): 0.5 ** 0.5,
            ("W0", "E1"): 0.5 ** 0.5,
        }
    )
    return sdict

Let’s create our model mapping:

In [ ]:
models = {
    "straight": straight,
    "bend_euler": bend_euler,
    "mmi1x2": mmi1x2,
}

We can now import the yaml netlist:

In [ ]:
mzi = sax.circuit_from_yaml(netlist, models=models)

In [ ]:
wl = jnp.linspace(1.5, 1.6, 1000)
params = sax.get_params(mzi)
params = sax.set_params(params, wl=wl)
transmission = jnp.abs(mzi(**params)["o1", "o2"]) ** 2
plt.plot(wl * 1e3, transmission)
plt.xlabel("λ [nm]")
plt.ylabel("T")
plt.show()

In [ ]:
mzi = sax.circuit_from_gdsfactory(mzi_component, models=models)
wl = jnp.linspace(1.5, 1.6, 1000)
params = sax.get_params(mzi)
params = sax.set_params(params, wl=wl)
transmission = jnp.abs(mzi(**params)["W0", "E0"]) ** 2
plt.plot(wl * 1e3, transmission)
plt.xlabel("λ [nm]")
plt.ylabel("T")
plt.show()
